In [14]:
import requests
import json
#import gmaps - future development
#import matplotlib - future development
import pandas as pd
from config import ABUSEIPDB_API_KEY
from config import VT_API_KEY
from config import VOID_API_KEY
#from config import GMAPS_API_KEY - map development

In [15]:
#Receive IP for analysis
ip = input('')

# Defining the api-endpoints
abuse_url = 'https://api.abuseipdb.com/api/v2/check'
vt_url = (f'https://www.virustotal.com/api/v3/ip_addresses/{ip}')
void_url = (f'https://endpoint.apivoid.com/iprep/v1/pay-as-you-go/?key={VOID_API_KEY}&ip={ip}')


abuse_querystring = {
    'ipAddress': ip,
    'maxAgeInDays': '365'
}

abuse_headers = {
    'Accept': 'application/json',
    'Key': ABUSEIPDB_API_KEY
}

vt_querystring = {
    'ip': ip
}

vt_headers = {
    'Accept': 'application/json',
    'X-APIKey': VT_API_KEY
}

abuse_response = requests.request(method='GET', url=abuse_url, headers=abuse_headers, params=abuse_querystring)
vt_response = requests.request(method='GET', url=vt_url, headers=vt_headers, params=vt_querystring)
void_response = requests.request(method='GET', url=void_url)

# Formatted output
abuse_decodedResponse = json.loads(abuse_response.text)
print (json.dumps(abuse_decodedResponse, sort_keys=True, indent=4))

vt_decodedResponse = json.loads(vt_response.text)
print (json.dumps(vt_decodedResponse, sort_keys=True, indent=4))

void_decodedResponse = json.loads(void_response.text)
print (json.dumps(void_decodedResponse, sort_keys=True, indent=4))

8.8.8.8
{
    "data": {
        "abuseConfidenceScore": 0,
        "countryCode": "US",
        "domain": "google.com",
        "hostnames": [
            "dns.google"
        ],
        "ipAddress": "8.8.8.8",
        "ipVersion": 4,
        "isPublic": true,
        "isWhitelisted": true,
        "isp": "Google LLC",
        "lastReportedAt": "2021-03-18T12:54:51+00:00",
        "numDistinctUsers": 9,
        "totalReports": 14,
        "usageType": "Data Center/Web Hosting/Transit"
    }
}
{
    "data": {
        "attributes": {
            "as_owner": "GOOGLE",
            "asn": 15169,
            "continent": "NA",
            "country": "US",
            "jarm": "29d3fd00029d29d00042d43d00041d598ac0c1012db967bb1ad0ff2491b3ae",
            "last_analysis_results": {
                "ADMINUSLabs": {
                    "category": "harmless",
                    "engine_name": "ADMINUSLabs",
                    "method": "blacklist",
                    "result": "clean"
         

In [16]:
#Variable Madness (Define Variables)
address = (abuse_decodedResponse['data']['ipAddress'])
confidence = (abuse_decodedResponse['data']['abuseConfidenceScore'])
country = (abuse_decodedResponse['data']['countryCode'])
hostname = (abuse_decodedResponse['data']['hostnames'])
isp = (abuse_decodedResponse['data']['isp'])
whitelist = (abuse_decodedResponse['data']['isWhitelisted'])
totalreports = (abuse_decodedResponse['data']['totalReports'])
usage = (abuse_decodedResponse['data']['usageType'])

harmless = (vt_decodedResponse['data']['attributes']['last_analysis_stats']['harmless'])
malicious = (vt_decodedResponse['data']['attributes']['last_analysis_stats']['malicious'])
suspicious = (vt_decodedResponse['data']['attributes']['last_analysis_stats']['suspicious'])

tor = (void_decodedResponse['data']['report']['anonymity']['is_tor'])
vpn = (void_decodedResponse['data']['report']['anonymity']['is_vpn'])
void_detection = (void_decodedResponse['data']['report']['blacklists']['detection_rate'])
region = (void_decodedResponse['data']['report']['information']['region_name'])
city = (void_decodedResponse['data']['report']['information']['city_name'])
latitude = (void_decodedResponse['data']['report']['information']['latitude'])
longitude = (void_decodedResponse['data']['report']['information']['longitude'])


In [17]:
#Make Pandas Dataframe look "crispy"
ip_data = {'Category':  ['IP Address',
                         'ISP',
                         'Country',
                         'Region / State',
                         'City',
                         'Whitelisted',
                         'Total Times Reported',
                         'Usage',
                         'Abuse IPDB Score',
                         'VT Considered Harmless',
                         'VT Considered Malicious',
                         'VT Considered Suspicious',
                         'TOR',
                         'VPN',
                         'API VOID Detection Rate'
                         ],
               'Values': [address,
                          isp,
                          country,
                          region,
                          city,
                          whitelist,
                          totalreports,
                          usage,
                          confidence,
                          harmless,
                          malicious,
                          suspicious,
                          tor,
                          vpn,
                          void_detection,
                        ]
              }

#Print out dataframe (plus some formatting)
ip_results = pd.DataFrame (ip_data, columns = ['Category','Values'])
ip_results = ip_results.set_index("Category", inplace = False)
ip_results = ip_results.style.set_caption("OSIPS Results")  

ip_results

,Values
Category,
IP Address,8.8.8.8
ISP,Google LLC
Country,US
Region / State,California
City,Mountain View
Whitelisted,True
Total Times Reported,14
Usage,Data Center/Web Hosting/Transit
Abuse IPDB Score,0
